d = Degree
r = Rank
n = Dimension
k = Subspace dimension
s = k-r
$$\delta\text{ = degree of the morphism}$$ 


In [20]:
def multiply_motives(motive1, motive2):
    len1 = len(motive1)
    len2 = len(motive2)
    total_len = len1+len2-1
    result = [0]*total_len
    while len(motive1)<total_len:
        motive1.append(0)
    while len(motive2)<total_len:
        motive2.append(0)
    for i in range(len1):
        for j in range(len2):
            result[i+j] +=  motive1[i]*motive2[j]
    return result

def add_motives(motive1, motive2):
    total_len = max(len(motive1), len(motive2))
    while len(motive1)<total_len:
        motive1.append(0)
    while len(motive2)<total_len:
        motive2.append(0)
    result = [0]*total_len
    for i in range(total_len):
        result[i] += motive1[i]+motive2[i]
    return result

The folowing function bia_bir_decomp(r,k,delta) calcuates $$[\bar{Q_\delta}]$$

In [23]:
import numpy as np
import itertools
def bia_bir_decomp(r,k,delta):
    s = k-r
    return [prop_4_point_15(r,s,delta,i) for i in range(k*delta+r*(k-r)+1)]
    
def computation1(a,b,c,s):
    total = 0
    i = 0
    while i<s:
        total += a[i]+c[i]*(1-b[i]+b[i+1])
        i+=1
    return total

# in order to calculate the number of sequences that satisfy 
# the equation of proposition 4.15 we enumerate all possible
# weakly sequences through a bijection to combinations.
# this however is exponential in its execution.
def prop_4_point_15(r,s,delta, i):
    total = 0
    for steps in itertools.combinations(range(r+s),r):
        c = []
        level = 0
        path_j = 0
        while path_j < r+s:
            if level<r and steps[level]==path_j:
                level += 1
                path_j+=1
            else:
                c.append(level)
                path_j+=1
        for steps_b in itertools.combinations(range(2*s-1+delta),delta):
            ab = []
            level_b = 0
            path_b_j = 0
            while path_b_j < delta+2*s-1:
                if level_b<delta and steps_b[level_b]==path_b_j:
                    level_b += 1
                    path_b_j+=1
                else:
                    ab.append(level_b)
                    path_b_j+=1
            a = ab[::2]
            b = [0]+ab[1::2]+[delta]
            if computation1(a,b,c,s)==i:
                total+=1
    return total


The folowing calculates
$$\Omega_{d-\delta}=\Sigma_{m\in Z^s, |m|=d-\delta}\frac{(1-L^{m_1+1})..(1-L^{m_s+1})}{(1-L)^s}L^{d_m}$$
$$d_m=\Sigma^s_i=0(i-1)m_i$$

In [24]:
def prop_4_point_13(r, k, w):
    s = k-r
    result = [0]
    for combination in itertools.product(range(w+1),repeat = s): 
        total = sum(combination)
        motive = [1]
        if total == w:
            for m_i in combination:
                motive_mult = [1]*(m_i+1)
                motive = multiply_motives(motive_mult, motive)
            d_m = computation_2(combination)+1
            other_motive = [0]*d_m
            other_motive[d_m-1] = 1
            motive = multiply_motives(motive, other_motive)
            result =add_motives(result, motive)
    return result

def computation_2(combination):
    total = 0
    i = 1
    for m in combination:
        total += (i-1)*m
        i+=1
    return total
        


The following calculates upside down Omega and finaly the motive of the space

In [27]:
def theorem_4_point_14(r, k,d):
    s = k-r
    result = [0]
    for j in range(d+1):
        motive = bia_bir_decomp(r,k,d-j)
        omega = recursion_16(r,k,j)
        result = add_motives(result,multiply_motives(motive,omega))
    return result
    
def recursion_16(r,k,j):
    i = 0
    if j == 0:
        return [1]
    motive = [0]
    while i<=j-1:
        motive_mult = multiply_motives(recursion_16(r,k,i),prop_4_point_13(r, k,j-i))
        motive = add_motives(motive, motive_mult)
        i+=1
    return multiply_motives(motive, [-1])


The 5 motives of the spaces $$[Rat_d(\mathbb{P}^1,\mathbb{G}(2,5))], d\in\{0,..,4\}$$

In [28]:
for i in range(5):
    print(theorem_4_point_14(2, 5,i))

[1, 1, 2, 2, 2, 1, 1]
[0, -1, -2, -3, -3, -2, 0, 2, 3, 3, 2, 1]
[0, 0, 0, 1, 1, 1, -1, -3, -5, -5, -3, 0, 3, 4, 4, 2, 1]
[0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 1, -2, -5, -7, -6, -3, 1, 4, 5, 4, 2, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 2, 0, -4, -7, -8, -6, -2, 2, 5, 5, 4, 2, 1]
